# **EDUARDO PEREZ CHAVARRIA. FT17**
## Proyecto individual
### FE. Análisis de sentimientos



In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import csv

import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

from langdetect import detect

## **1. Analisis de sentimientos**

### **Importamos el archivo ya curado que contiene las reviews de usuarios**

In [2]:
data_curado = "bases/users_review_curado.csv"

# Importar el archivo CSV como DataFrame
df_usrev = pd.read_csv(data_curado, encoding="utf-8")

# Mostrar información del DataFrame importado
print(df_usrev.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58400 entries, 0 to 58399
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    58400 non-null  object
 1   item_id    58400 non-null  int64 
 2   recommend  58400 non-null  bool  
 3   review     58400 non-null  object
 4   year       58400 non-null  object
dtypes: bool(1), int64(1), object(3)
memory usage: 1.8+ MB
None



### **Realizamos el análisis de sentimientos**

In [3]:
# Descarga los recursos necesarios de NLTK
nltk.download("vader_lexicon")

# Inicializa el analizador de sentimientos de NLTK
sia = SentimentIntensityAnalyzer()

# Función para asignar valores según la escala
def obtener_puntaje_sentimiento(texto):
    if pd.isnull(texto) or texto == "":
        return 1  # Retorna neutral si está vacío o es NaN
    elif isinstance(texto, str):
        # Analiza el sentimiento usando Vader de NLTK
        vader_sentimiento = sia.polarity_scores(texto)
        vader_puntaje_compuesto = vader_sentimiento["compound"]

        # Analiza el sentimiento usando TextBlob
        textblob_analysis = TextBlob(texto)
        polarity = textblob_analysis.sentiment.polarity

        # Aplica la escala para determinar el puntaje final
        if vader_puntaje_compuesto >= -0.05 and polarity >= -0.2 and polarity <= 0.2:
            return 1  # Neutral
        elif vader_puntaje_compuesto >= -0.05 or polarity > 0.2:
            return 2  # Buen puntaje
        elif vader_puntaje_compuesto <= -0.05 or polarity < -0.2:
            return 0  # Mal puntaje
    return 1  # Retorna neutral para otros casos

# Convierte la columna "review" a str si no lo es
df_usrev["review"] = df_usrev["review"].astype(str)

# Aplica la función obtener_puntaje_sentimiento a la columna "review"
df_usrev["Sentiment_Score"] = df_usrev["review"].apply(obtener_puntaje_sentimiento)

# Visualiza el DataFrame
print(df_usrev)


[nltk_data] Downloading package vader_lexicon to C:\Users\Legion 5
[nltk_data]     3060\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                 user_id  item_id  recommend  \
0      76561197970982479     1250       True   
1      76561197970982479    22200       True   
2      76561197970982479    43110       True   
3                js41637   251610       True   
4                js41637   227300       True   
...                  ...      ...        ...   
58395  76561198312638244       70       True   
58396  76561198312638244   362890       True   
58397        LydiaMorley   273110       True   
58398        LydiaMorley      730       True   
58399        LydiaMorley      440       True   

                                                  review      year  \
0      Simple yet with great replayability. In my opi...    2011.0   
1                   It's unique and worth a playthrough.    2011.0   
2      Great atmosphere. The gunplay can be a bit chu...    2011.0   
3      I know what you think when you see this title ...    2014.0   
4      For a simple (it's actually not all that simpl...    2013.0   
...

### **Explorar en qué idioma están las reseña**

Vamos a explorar en qué idioma están las reseñas para hacernos una idea de la composición de la columna. En teoría vader funciona bien para cualquier idioma aunque fue entrenado principalmente con texto en inglés.

In [4]:
# Función para detectar el idioma de la reseña
def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return "No detectado"  # Si no se puede detectar el idioma, se marca como "No detectado"

# Aplicar la función a la columna "review" y crear una nueva columna "idioma"
df_usrev["idioma"] = df_usrev["review"].apply(detectar_idioma)

df_usrev

,user_id,item_id,recommend,review,year,Sentiment_Score,idioma
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011.0,1,en
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011.0,2,en
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011.0,1,en
3,js41637,251610,True,I know what you think when you see this title ...,2014.0,1,en
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013.0,1,en
...,...,...,...,...,...,...,...
58395,76561198312638244,70,True,a must have classic from steam definitely wort...,Sin dato,2,en
58396,76561198312638244,362890,True,this game is a perfect remake of the original ...,Sin dato,1,en
58397,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,Sin dato,1,en
58398,LydiaMorley,730,True,:D,Sin dato,2,de


### **Valores unicos para los idiomas**

In [5]:
conteo_idiomas = df_usrev["idioma"].value_counts()
print(conteo_idiomas)


idioma
en              45085
pt               2155
es               1255
de               1132
so               1003
af                750
th                707
tl                573
No detectado      543
cy                444
da                399
no                366
nl                304
ca                258
ro                246
it                246
pl                240
id                239
ko                234
fr                226
ru                213
zh-cn             208
sw                187
et                169
tr                163
hr                141
sl                138
vi                119
sq                107
fi                101
hu                 94
sv                 78
sk                 74
lt                 42
cs                 37
lv                 37
ja                 36
bg                 19
uk                 13
zh-tw               9
mk                  7
ar                  2
ta                  1
Name: count, dtype: int64


Notamos que casi el 80% de las reseñas están en inglés. Por lo que el análisis sigue siendo bastante valido. Vamos a explorar los casos donde no se detectó el idioma y aun así realizó análisis de sentimientos

In [6]:
# Filtra las filas donde el idioma es "No detectado" y muestra las filas relevantes
sin_idioma_detectado = df_usrev[df_usrev["idioma"] == "No detectado"]
resultados_analisis = sin_idioma_detectado[["user_id", "item_id", "recommend", "review", "year", "Sentiment_Score", "idioma"]]

# Visualiza los análisis de sentimiento para las reseñas sin idioma detectado
print(resultados_analisis)

# Exporta los resultados a un archivo de texto
resultados_analisis.to_csv("resultados_analisis_sin_idioma_detectado.txt", sep="\t", index=False)


                 user_id  item_id  recommend  \
614    76561198070263209      570       True   
632          NaruseAiria   108600       True   
638    76561198044678148   495890       True   
752          XABLAUziitu      440       True   
869            Swish0009   213850       True   
...                  ...      ...        ...   
57790        hfeygugbuhg    49520       True   
57959        hlebteampro   223390       True   
57976              42456   304930       True   
58104             ButNae   113400      False   
58343           AusBacon   369290      False   

                                                  review      year  \
614                                                         2013.0   
632    ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...    2014.0   
638    My godess~░░░░░░░█▐▓▓░████▄▄▄█▀▄▓▓▓▌█ ░░░░░▄█▌...  Sin dato   
752                                     ♥♥♥♥ pra ♥♥♥♥♥♥♥    2013.0   
869                                                ♥♥♥♥!    2014.0   
...

Se puede observar que varios de los casos contienen simbolos que pueden tomarse como evaluaciones positivas. Por ello decidimos conservarlos. 

In [7]:
df_usrev[["review", "Sentiment_Score"]]


,review,Sentiment_Score
0,Simple yet with great replayability. In my opi...,1
1,It's unique and worth a playthrough.,2
2,Great atmosphere. The gunplay can be a bit chu...,1
3,I know what you think when you see this title ...,1
4,For a simple (it's actually not all that simpl...,1
...,...,...
58395,a must have classic from steam definitely wort...,2
58396,this game is a perfect remake of the original ...,1
58397,had so much fun plaing this and collecting res...,1
58398,:D,2


### **Frencuencias de la codificacion de analisis**

In [8]:
frecuencias = df_usrev["Sentiment_Score"].value_counts()
print(frecuencias)


Sentiment_Score
1    29732
2    19904
0     8764
Name: count, dtype: int64


### **Reseñas no detectadas **

con el txt que exportamos antes podemos notar que hay varias respuestas tipo <3, corazones, 100/10, 😄, 9/11, ♥, :), 💋, < 3, ★★★★★ que indican que la reseña es positiva, vamos a cambiarlas cuando haya ese tipo de respuesta

In [9]:
def asignar_sentimiento_especial(texto):
    # Expresiones que indican un sentimiento positivo no neutro ni negativo
    expresiones_positivas = [r"< ?3{3,}", r"♥{3,}", r":\){3,}", r"★{3,}"]

    # Expresiones que indican una calificación alta
    expresiones_calificacion = [r"\d{3,5}/10", r"[8-9]/10|[1-9][0-9]{1,3}", r"[8-9]/[10-1000]"]

    for exp in expresiones_positivas + expresiones_calificacion:
        if re.search(exp, texto):
            return 2  # Sentimiento positivo alto

    return 1  # Si no se encuentra ninguna expresión especial, se asigna neutral

# Crear una máscara para las filas con "idioma" igual a "No detectado"
mask_no_detectado = df_usrev["idioma"] == "No detectado"

# Aplicar la función a las filas con "idioma" igual a "No detectado"
df_usrev.loc[mask_no_detectado, "Sentiment_Score"] = df_usrev.loc[mask_no_detectado, "review"].apply(asignar_sentimiento_especial)

# Visualizar los cambios en el DataFrame
print(df_usrev)

# Exportar a un archivo de texto solo las filas con "idioma" igual a "No detectado"
resultados_analisis = df_usrev[mask_no_detectado]
resultados_analisis.to_csv("resultados_analisis_sentimiento_especial.txt", sep="\t", index=False)


                 user_id  item_id  recommend  \
0      76561197970982479     1250       True   
1      76561197970982479    22200       True   
2      76561197970982479    43110       True   
3                js41637   251610       True   
4                js41637   227300       True   
...                  ...      ...        ...   
58395  76561198312638244       70       True   
58396  76561198312638244   362890       True   
58397        LydiaMorley   273110       True   
58398        LydiaMorley      730       True   
58399        LydiaMorley      440       True   

                                                  review      year  \
0      Simple yet with great replayability. In my opi...    2011.0   
1                   It's unique and worth a playthrough.    2011.0   
2      Great atmosphere. The gunplay can be a bit chu...    2011.0   
3      I know what you think when you see this title ...    2014.0   
4      For a simple (it's actually not all that simpl...    2013.0   
...

Vamos a ver cómo han cambiado los valores unicos. Antes del cambio eran estos

* Sentiment_Score
* 1    29735
* 2    19901
* 0     8764
* Name: count, dtype: int64**

In [10]:
frecuencias = df_usrev["Sentiment_Score"].value_counts()
print(frecuencias)

Sentiment_Score
1    29558
2    20087
0     8755
Name: count, dtype: int64


Se aprecian pocos cambios significativos al implementar esa medida de cuidado. Vamos a eliminar la columna de idioma que ya no utilizaremos

In [11]:
df_usrev.drop(columns=["idioma"], inplace=True)

## **Observamos el df final**

In [12]:
df_usrev

,user_id,item_id,recommend,review,year,Sentiment_Score
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011.0,1
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011.0,2
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011.0,1
3,js41637,251610,True,I know what you think when you see this title ...,2014.0,1
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013.0,1
...,...,...,...,...,...,...
58395,76561198312638244,70,True,a must have classic from steam definitely wort...,Sin dato,2
58396,76561198312638244,362890,True,this game is a perfect remake of the original ...,Sin dato,1
58397,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,Sin dato,1
58398,LydiaMorley,730,True,:D,Sin dato,2


## **Guardamos la base**

In [13]:
data_curado_sentiments = "bases/users_review_curado_sentiments.parquet"

# Guardar el DataFrame en formato Parquet con comillas dobles
df_usrev.to_parquet(data_curado_sentiments, index=False, engine="pyarrow", compression="gzip")

print(f"Has guardado tu base en formato Parquet con comillas dobles: {data_curado_sentiments}")


c:\Users\Legion 5 3060\.conda\envs\SoyHenryProyectconda\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Has guardado tu base en formato Parquet con comillas dobles: bases/users_review_curado_sentiments.parquet


In [14]:
# Cargar el archivo Parquet de nuevo
df_importado = pd.read_parquet("bases/users_review_curado_sentiments.parquet")

# Verificar que se haya cargado correctamente
df_importado


,user_id,item_id,recommend,review,year,Sentiment_Score
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011.0,1
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011.0,2
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011.0,1
3,js41637,251610,True,I know what you think when you see this title ...,2014.0,1
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013.0,1
...,...,...,...,...,...,...
58395,76561198312638244,70,True,a must have classic from steam definitely wort...,Sin dato,2
58396,76561198312638244,362890,True,this game is a perfect remake of the original ...,Sin dato,1
58397,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,Sin dato,1
58398,LydiaMorley,730,True,:D,Sin dato,2
